In [1]:
from visualizer import calibration
import visual

from PyQt5 import QtCore
import tifffile
from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
from glob import glob
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import napari
from napari import Viewer
import os
import zarr
from skimage.registration import phase_cross_correlation
import cv2 as cv
from skimage import io

import cupy as cp 
import imagej

import waveorder as wo
from waveorder.io.multipagetiff import  MicromanagerOmeTiffReader
from aicsimageio import AICSImage,imread, imread_dask

%gui qt 
%matplotlib inline
plt.style.use('dark_background')


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [ ]:
# main_folder = r'H:\Data\PSF\1um\220507_190nmSteps_1'


main_folder = r'D:\Matlab Magic\Extended-Volume-3D-deconvolution-master\Extended-Volume-3D-deconvolution-master\10x40_0005'
processed_files = main_folder + '/'+'processed_files'
if not os.path.exists(processed_files):
    os.makedirs(processed_files)
# main_folder = r'D:\Matlab Magic\Extended-Volume-3D-deconvolution-master\Extended-Volume-3D-deconvolution-master'
# file_extension = main_folder+ '/'+'zstack' + '*.tif'
file_extension = main_folder +'/' + '*.tif'
offsets_file = os.path.join(main_folder,"offsets.csv")
file_names = sorted(glob(file_extension),key=alphanumeric_key)

In [3]:
stack = [tifffile.imread(file_names[files]) for files in range(len(file_names))]
da_stack = da.array(stack)

In [4]:
#Scope Parameters MBL M25
# FOV = 50e-6
cam_px = 6.0e-6
totalmag = 15.75
px_size_img = cam_px/totalmag
zstep = 2e-6
z_scale = zstep/px_size_img


#Scope Parameters UCSC M25
# FOV = 50e-6
# cam_px = 5.86e-6
# totalmag = 37.5
# px_size_img = cam_px/totalmag
# zstep = 2e-6
# z_scale = zstep/px_size_img

print(px_size_img)
print(zstep/cam_px)
print(z_scale)

3.8095238095238096e-07
0.3333333333333333
5.25


In [5]:
viewer=napari.Viewer()
viewer.add_image(da_stack,scale=[z_scale,1,1])

<Image layer 'da_stack' at 0x255b1f22d90>

c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_struct__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array_interface__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2443: FutureWarning: Private attribute access ('ClippingPlaneList.__array__') in this context (e.g. inside a plugin widget or dock widget) is deprecated and will be unavailable in version 0.5.0
  a1, a2 = asarray(a1), asarray(a2)
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-

### Get Calibration from Bead's Files

In [14]:
# Generate our own shifted datset
with cp.cuda.Device(0):
    test = stack
    t,c,h,w  = test.shape
    np_mip =[]
    for i in range(c):
        cp_stack = cp.array(test[:,i,:,:])
        cp_mip = cp.max(cp_stack, axis=0)
        np_mip.append(cp.asnumpy(cp_mip))
        cp_mip = None
        cp_stack=None
    np_mip = np.array(np_mip)
    np_mip = np.expand_dims(np_mip,axis=0)
    

In [17]:
# Show the maximum intensity projection
viewer = napari.Viewer()
viewer.add_image(np_mip, name='mip')

<Image layer 'mip' at 0x20daf077b20>

c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [18]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)


[[  0.          12.         241.19375301 352.08221862]
 [  0.          12.         241.19375301 441.60962625]
 [  0.          12.         341.60962913 441.60962625]
 [  0.          12.         341.60962913 352.08221862]]
[[241 352]
 [342 442]]
(2, 2)


In [19]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
np_mip_8bit =visual.im_bit_convert(np_mip,bit=8,norm=True)
min_val, max_val = template_box
t,c = template[0,0:2]
crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
viewer.add_image(crop_img,name='crop')

<Image layer 'crop' at 0x20dc61578b0>

In [20]:
coord =[]
w,h = template.shape  
methods = 'cv.TM_CCOEFF'
print(w,h)
t,c,h,w = np_mip_8bit.shape

for i in range(c):
    method = eval(methods) # This method parses expression and runs ()
    # Apply template Matching
    # Slide through image and compare template patches.
    # Comparison of best matches is foudn as global minn in SQDIFF or max in CCORR or CCOEF.
    res = cv.matchTemplate(np_mip_8bit[0,i,:,:],crop_img,method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
        
    bottom_right = (top_left[0] + w, top_left[1] + h)
    #Output would contain the Top left corner and bottom right in case of need to use this rectangle
    coord.append(((top_left[1],top_left[0]),(bottom_right[1],bottom_right[0])))

#For the bead calibration we only use the TL points for alignment
TL_coords = np.array(coord)
TL_coords= TL_coords[0:25,0]

#Coordinates need to then be subtracted to the reference frame (i.e t,c for which rectangle was drawn)
res = []
ref_cam_index = 0
# diff =[]
for i,coordinate in enumerate(TL_coords):
    val = tuple(map(lambda i, j: i - j, TL_coords[ref_cam_index],coordinate))
    res.append(val)
    #If previous values available we can find teh difference between them
    # diff.append(tuple(map(lambda i, j: i + j, val,shift_stack_coord[i])))
# print(diff)
print(res)
# print(shift_stack_coord)
offset_coordinates = np.array(res)

#Save the Offsets in the folder
np.savetxt(offsets_file, offset_coordinates, delimiter=',')

2 4
[(0, 0), (-34, -33), (5, -58), (-11, -95), (-28, -67), (70, -14), (11, -13), (29, -48), (-16, -99), (-16, -23), (6, 3), (15, -55), (33, -50), (-4, -60), (25, -43), (13, -45), (17, -52), (31, -61), (-10, -45), (16, -53), (20, -36), (55, -12), (50, -42), (17, -31), (38, -49)]


In [21]:
#Aligning Max Projection to check if coordinates are good

from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset
with cp.cuda.Device(0):
    mip_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(np_mip.shape)
    for i in range(c):
        cp_stack = cp.array(np_mip[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        mip_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    mip_aligned = np.array(mip_aligned)
mip_aligned =  np.moveaxis(mip_aligned,0,1)

#Convert to Dask Array
mip_aligned = da.from_array(mip_aligned)
viewer.add_image(mip_aligned, name='MIP_aligned',scale = [z_scale,1,1])

(25, 3)
(1, 25, 608, 808)


<Image layer 'MIP_aligned' at 0x20dc98bc880>

In [22]:
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

#Convert to Dask Array
da_stack_align = da.from_array(stack_aligned)


(25, 3)
(316, 25, 608, 808)


In [23]:
viewer.add_image(da_stack_align, name='aligned_stack',scale = [z_scale,1,1])

<Image layer 'aligned_stack' at 0x20dc74b1730>

### Align Datasets with Known Offsets


In [8]:
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset

offset_coordinates = offsets
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

(25, 3)
(316, 25, 608, 808)


In [9]:
#Convert numpy to dask?
da_stack_align = da.from_array(stack_aligned)
print(da_stack_align)

dask.array<array, shape=(316, 25, 608, 808), dtype=uint16, chunksize=(79, 25, 76, 101), chunktype=numpy.ndarray>


In [10]:
viewer= napari.Viewer()

In [11]:
viewer.add_image(da_stack_align, name='dask_align',scale=[z_scale,1,1])

<Image layer 'dask_align' at 0x15260c2df10>

c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [20]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(da_stack_align,dtype='uint16'))
ij.ui().show('PSF', ij_stack2)

### SAVE TIFF ALIGNED

In [24]:
import tifffile

filename = 'aligned_stack' + '.ome.tif'
filepath_save = os.path.join(processed_files,filename)
print(filepath_save)
tifffile.imsave(filepath_save,stack_aligned)

H:\Data\PSF\200nm\220508_190nmSteps_postitive2negative_1/processed_files\aligned_stack.ome.tif


C:\Users\yoshi\AppData\Local\Temp\ipykernel_64764\4020059993.py:6: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(filepath_save,stack_aligned)
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())
c:\Users\yoshi\Anaconda3\envs\napari-m25\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


### CROP PSF TO ANALYZE


In [25]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)

[[151.          11.         292.00660599 360.55102745]
 [151.          11.         292.00660599 385.95745394]
 [151.          11.         318.62286231 385.95745394]
 [151.          11.         318.62286231 360.55102745]]
[[292 361]
 [319 386]]
(2, 2)


In [26]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
# np_mip_8bit =visual.im_bit_convert(np_mip,bit=8)
min_val, max_val = template_box
t,c = template[0,0:2]
print(t,c)
crop_img = da_stack_align[:,:, min_val[0]:max_val[0],min_val[1]:max_val[1]]
# crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
viewer.add_image(crop_img,name='crop')

151 11


<Image layer 'crop' at 0x20dce891190>

### Image J Part

In [28]:
def start_imagej():
    import imagej
    global ij
    ij = imagej.init( mode='interactive')
    # path = r'C:\Users\yoshi\Documents\fiji-win64\Fiji.app'
    # ij = imagej.init(path,mode='interactive')
    ij.ui().showUI()
    print(ij.getVersion())

QtCore.QTimer.singleShot(0, start_imagej)

In [39]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(crop_img))
ij.ui().show('PSF', ij_stack2)
# psf_window =ij.WindowManager.getWindow('PSF')
# ij.WindowManager.setWindow(psf_window)

In [40]:
macro = """
run("Reslice [/]...", "output=0.19 start=Top avoid");
run("Z Project...", "projection=[Max Intensity]");
Stack.setDisplayMode("grayscale");
run("Make Montage...", "columns=25 rows=1 scale=1 label");
rename("montage");
"""
result = ij.py.run_macro(macro)

array(None, dtype=object)

In [41]:
b =ij.WindowManager.getWindow('montage')
ij.WindowManager.setWindow(b)
montage = ij.WindowManager.getCurrentImage()
np_montage = ij.py.from_java(montage)
viewer.add_image(np_montage)

filename = 'montage' + '.ome.tif'
filepath_save = os.path.join(processed_files,filename)
print(filepath_save)
tifffile.imwrite(filepath_save,np_montage)

H:\Data\PSF\200nm\220508_190nmSteps_postitive2negative_1/processed_files\montage.ome.tif


: 

### Save PSF

In [27]:
filename = 'psf_crop' + '.ome.tif'
filepath_save = os.path.join(processed_files,filename)
print(filepath_save)
tifffile.imwrite(filepath_save,crop_img)

H:\Data\PSF\200nm\220508_190nmSteps_postitive2negative_1\psf_crop.ome.tif


C:\Users\yoshi\AppData\Local\Temp\ipykernel_64764\638195454.py:4: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(filepath_save,crop_img)


### OPEN STACK


In [5]:
filename = 'aligned_stack' + '.ome.tif'
filepath_save = os.path.join(main_folder,filename)
stack_aligned = tifffile.imread(filepath_save)
da_stack_align = da.from_array(stack_aligned)
viewer.add_image(da_stack_align, name='aligned_stack',scale = [z_scale,1,1])

FileNotFoundError: [Errno 2] No such file or directory: 'H:\\Data\\PSF\\200nm\\220508_190nmSteps_postitive2negative_1\\aligned_stack.ome.tif'

In [70]:
b =ij.WindowManager.getWindow('montage')
ij.WindowManager.setWindow(b)

In [53]:
ij.WindowManager.setWindow(b)
a =ij.WindowManager.getCurrentImage()
anp = ij.py.from_java(a)
print(anp.shape)
anp2 = anp[:,:,:,:,np.newaxis]
print(anp2.shape)
anp2 = np.swapaxes(anp,1,4)
anp2.shape


(385, 9, 8, 25)


TypeError: invalid indexer array, does not have integer dtype: array(None, dtype=object)

In [48]:
print(anp2.shape)
visual.image_stack_viewer(anp2)

(385, 25, 10, 10)


interactive(children=(IntSlider(value=0, description='stack_idx_1', max=384), IntSlider(value=0, description='…

<function visual.image_stack_viewer.<locals>.interact_plot_4D(stack_idx_1, stack_idx_2)>

SyntaxError: invalid syntax (1059654506.py, line 1)

In [4]:
# def start_imagej():
#     global ij
#     import imagej
#     ij = imagej.init(r'C:\Users\Callisto\Documents\edhirata\software\Fiji.app',mode='interactive')
#     ij.ui().showUI()
#     print(ij.getVersion())
    
# QtCore.QTimer.singleShot(0, start_imagej)

sys:1: RuntimeWarning: coroutine 'get_msg' was never awaited


In [19]:
# os.environ['JAVA_HOME'] = r'C:\Users\Callisto\Anaconda3\envs\napari-pymmcore'